Nombre: Victor Joel Pinargote<br>
GitHub: https://github.com/kmotillo/03MAIR_ALGORITMOS_OPTIMIZACION/blob/main/AG3/TSP_GRASP_ULTIMO.ipynb <br>
Colab: https://drive.google.com/file/d/1sGzJLYaCYr_3pnjrYlbXaZsJMYZHpzzy/view?usp=sharing <br>

# Cargamos todas la librerias

In [1]:
!pip install tsplib95
!pip install pandas

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from time import time
import copy
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp


# Descargamos el fichero de datos(Matriz de distancias)
### Carga de datos y generación de objeto problem

In [3]:

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion : 
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#file = "swiss42.tsp"
#file = "fri26.tsp" ;
#file = "dantzig42.tsp" ;

def loadProblem(file):
    url = "http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/"+file+""
    urllib.request.urlretrieve(url, file) 

    # Aquí Cargamos el problema en la variable problem
    problem = tsplib95.load(file)

    #Nodos
    Nodos = list(problem.get_nodes())

    #Aristas
    Aristas = list(problem.get_edges())

    #Distancia entre nodos
    problem.get_weight(1, 1)

    n = problem.dimension

    #Obtengo los pesos mas pequeños
    nodosPesos = [(i, j, problem.get_weight(i, j)) for i,j in problem.get_edges()]
    nodosPesos = np.array(nodosPesos)
    nodosPesos = nodosPesos[nodosPesos[:,2].argsort()]
    nodosPesos = nodosPesos[nodosPesos[:,2]>0]
    nodoMasPeque = nodosPesos[0]
    return n,problem,nodoMasPeque

# Funcion Objetivo Viajante de Comercio (TS)

In [4]:
def foViajero(solucion, problem):
    foSum = 0
    for i in range(0, len(solucion)-1):
        foSum+=problem.get_weight(solucion[i] ,solucion[i+1])
    foSum+=problem.get_weight(solucion[len(solucion)-1], solucion[0])
    return foSum

# Fase de construccion
Aquí la idea es Construir una solución medianamente aceptable, para lo cual aplico los criterios de Greedy, Randomize, Adaptative Search Procedure (GRASP): Seleccionando las dos primeras ciudades vorazmente, en este caso las que tengan menor distancia, luego me creo una Lista Reducidad e Candidatos que llamaré LRC la misma que escogerá 3 elementos aleatoreamente que no sean parte de la solución para luego quedarme con uno de ellos, `solo uno`, me quedaré con el que mejore  mi funcion de fitness (el que minimize la distancia). Repetire este proceso de llamar a la LRC() hasta que complete mi solución que tendrá una longitud de `n-2`, puesto que las dos primeras ciudades ya las agregué a la solucion vorazmente.

In [5]:
def LRC(solInt, problem):
    _LRC = []
    notInSol = set(list(problem.get_nodes()))
    notInSol = notInSol.difference(solInt)
    if (len(notInSol)<3):
        _LRC = random.sample(notInSol, k=len(notInSol))
    else:
        _LRC = random.sample(notInSol, k=3)
    return _LRC

In [6]:
def faseConstructiva(n, problem):
    sol = []
    # los 2 primeros puntos los calculo VORAZMENTE
    xy = nodoMasPeque
    x=xy[0]
    y=xy[1]
    sol.append(x)
    sol.append(y)
    
    contador = 0
    # llamo a la lista reducida de candidatos hasta completar la soluciión
    while contador < n-2:
        contador+=1
        _LRC = LRC(sol, problem)
        fo = np.inf
      
        for i in _LRC:
            sol.append(i)
            _fo = foViajero(sol, problem)
            sol.remove(i) 
            if _fo < fo:
                fo = _fo
                aux = i
                       
        sol.append(aux)
           
    return sol           

In [7]:
#solInicial = faseConstructiva(n, problem)
#foViajero(solInicial, problem)

Me creo una Heuristica donde construyo muchas soluciones `nSol` para luego quedarme con la mejor.

In [8]:
def xConstrucciones(n, problem, nSol):
    mejorSolucion=[]
    fo = np.inf
    for i in range(0, nSol):
        mejorSolucionTemp = faseConstructiva(n, problem)
        _fo = foViajero(mejorSolucionTemp, problem)
        if _fo < fo:
            fo = _fo
            mejorSolucion = copy.deepcopy(mejorSolucionTemp)
            
    return mejorSolucion

In [9]:
#solucionX = xConstrucciones(n, problem, 1000)
#foViajero(solucionX, problem)

# busqueda local

Aqui me he creado 3 metodos:

- local_search_FI_swap : estoy aplicando el criterio de SWAP y un First Improvement, donde la idea es intercambiar cada ciudad con las demás, pero cuando encuentre una mejora en dicho vecindario, utilizaré un `break` para salir de ese vecindario y continuar con el otro, de alli el first improvement, me quedo con primer vecino que mejore mi solución

- local_search_BI_swap : estoy aplicando el criterio de SWAP y un Best Improvement, donde la idea es intercambiar cada ciudad con las demás, a diferencia del first improvement no utilizaré un break `NO break` para salir de ese vecindario, por el contrario evaluaré a todos los vecinos y me quedaré con el mejor, de allí el birst improvement, me quedo con el mejor vecino.

- local_search_2_opt : estoy aplicando en criterio 2-opt, donde la idea es intercambiar cada ciudad con su vecina más cercana, este algoritmo es uno de los mas usados para el problema del viajero porque asegura una localidad fuerte.

Todos estos algoritmos estan implementados con un while para asegurar que solo se detengan cuando ya no mejore con ningun movimiento, es decir cuando encontramos un minimo local.


In [10]:
# Function: swap: Localidada debil 
def local_search_FI_swap(solucion, problem):
    fo = foViajero(solucion, problem)
    solLocal = copy.deepcopy(solucion)
    # notInSol = notInSol.difference(solInt)
    mejora = True
    # solucion = solInt
    saltos=0
    n = problem.dimension

    while (mejora):
        mejora = False
        
        for i in range(0, n):
            mejora = False
            elem1 = solLocal[i]
            for j in range(i+1, n):
              elem2 = solLocal[j]
              solLocal[i] = elem2
              solLocal[j] = elem1
              _fo = foViajero(solLocal, problem)
              
              #print("Sol:",i,j,solLocal,"FO:", _fo)
              if _fo < fo:
                #print(_fo)
                fo = _fo
                mejora = True 
                break
              else:
                solLocal[i] = elem1
                solLocal[j] = elem2
        saltos+=1
        
    return solLocal

# Function: swap: Localidada debil 
def local_search_BI_swap(solucion, problem):
    fo = foViajero(solucion, problem)
    solLocal = copy.deepcopy(solucion)
    # notInSol = notInSol.difference(solInt)
    mejora = True
    # solucion = solInt
    saltos=0
    n = problem.dimension
    mejorSolTemp = copy.deepcopy(solucion)

    while (mejora):
        mejora = False
        
        for i in range(0, n):
            mejora = False
            elem1 = solLocal[i]
            fo = foViajero(solLocal, problem)
            for j in range(i+1, n):
              elem2 = solLocal[j]
              solLocal[i] = elem2
              solLocal[j] = elem1
              _fo = foViajero(solLocal, problem)
              
              #print("Sol:",i,j,solLocal,"FO:", _fo)
              if _fo < fo:
                #print(_fo)
                fo = _fo
                mejorSolTemp = copy.deepcopy(solLocal)
                mejora = True

              solLocal[i] = elem1
              solLocal[j] = elem2
            
                
        saltos+=1
        
    return mejorSolTemp

# Function: 2-opt: Localidada debil 
def local_search_2_opt(solucion, problem):
    fo = foViajero(solucion, problem)
    solLocal = copy.deepcopy(solucion)
    # notInSol = notInSol.difference(solInt)
    mejora = True
    # solucion = solInt
    saltos=0
    n = problem.dimension

    while (mejora):
        mejora = False
        
        for i in range(0, n-1):
          mejora = False
          elem1 = solLocal[i]
          elem2 = solLocal[i+1]

          solLocal[i] = elem2
          solLocal[i+1] = elem1

          _fo = foViajero(solLocal, problem)
          #print("Sol:",i,solLocal,"FO:", _fo)
          if _fo < fo:
            #print(_fo)
            fo = _fo
            mejora = True 
          else:
            solLocal[i] = elem1
            solLocal[i+1] = elem2
        saltos+=1
        
    return solLocal  

Aquí me implemento una función `busquedaLocal` que recive la solucion generada por la fase de construcción y defino que metos de busqueda local puedo utilizar, en la variable `method=[0]`:
- 0 : usaremos los 3 metodos
- 1 : usaremos el 2-opt
- 2 : usaremos en SWAP con first improvement
- 3 : usaremos en SWAP con best improvement

he probado algunas combinaciones y la que moejor resultados me ha dado es la `method=[1,3]`

In [11]:
def busquedaLocal(solucionX, problem, method=[0]):
    fo = foViajero(solucionX, problem)
    mejorSolucion = copy.deepcopy(solucionX)
    mejora = True

    two_opt=False
    fi_swap=False
    bi_swap=False

    if 0 in method:
        two_opt=True
        fi_swap=True
        bi_swap=True
    if 1 in method:
        two_opt=True
    if 2 in method:
        fi_swap=True
    if 3 in method:
        bi_swap=True

    while (mejora):
        mejora = False
        if two_opt:
            mejorSolucion = local_search_2_opt(mejorSolucion, problem)
        if bi_swap:
            mejorSolucion = local_search_BI_swap(mejorSolucion, problem)
        if fi_swap:
            mejorSolucion = local_search_FI_swap(mejorSolucion, problem)

        _fo = foViajero(mejorSolucion, problem)
        if _fo < fo:
            fo = _fo
            mejora = True

    return mejorSolucion

Finalmente me genero otra Heuristica donde ejecuto `nInstancias` el algoritmo completo y me quedo con la mejor solución.

In [12]:
def instancias(n, problem, nInstancias=10):
    fo = np.inf
    bestSol = []

    for i in range(nInstancias):
        # Fase de Construccion
        solucionX = xConstrucciones(n, problem, 100)
        #Fase de Busqueda Local
        solFaseBusquedaLocal = busquedaLocal(solucionX, problem, method=[1,3])
        _fo = foViajero(solFaseBusquedaLocal, problem)
        #print("Solucion:",i,"FO:",_fo)
        if _fo < fo:
            fo = _fo
            bestSol = copy.deepcopy(solFaseBusquedaLocal)

    return bestSol


In [16]:
%%time
file = "swiss42.tsp"
nInstancias=200
n,problem,nodoMasPeque = loadProblem(file)
problemType = problem.edge_weight_type
if problemType == 'EXPLICIT':
    solInstancias = instancias(n, problem, nInstancias=100)
    fo = foViajero(solInstancias, problem)
    print("FO:", fo)
else:
    print("Instancia no valida")

FO: 1488
CPU times: user 9min 6s, sys: 1.03 s, total: 9min 7s
Wall time: 9min 8s


In [14]:
print(solInstancias)

[30, 28, 2, 27, 0, 7, 37, 15, 16, 14, 19, 13, 1, 3, 4, 6, 5, 26, 18, 12, 11, 25, 10, 8, 29, 22, 39, 9, 41, 23, 21, 40, 24, 38, 34, 33, 20, 35, 36, 17, 31, 32]


- Para la instancia `swiss42.tsp` la mejor solucion encontrada con el GRASP fue: [30, 28, 2, 27, 0, 7, 37, 15, 16, 14, 19, 13, 1, 3, 4, 6, 5, 26, 18, 12, 11, 25, 10, 8, 29, 22, 39, 9, 41, 23, 21, 40, 24, 38, 34, 33, 20, 35, 36, 17, 31, 32] con una `FO: 1488`

- Esta es la mejor solución que enconté para una instancia del problema `fri26.tsp : 962` que esta muy cercana a la mejor conocida: `fri26 : 937`